![MLU Logo](../data/MLU_Logo.png)

# <a name="0">Machine Learning Accelerator - Natural Language Processing - Lecture 3</a>

## Recurrent Neural Networks (RNNs) for the Product Review Problem - Classify Product Reviews as Positive or Not

In this exercise, we will learn how to use Recurrent Neural Networks. 

We will follow these steps:
1. <a href="#1">Reading the dataset</a>
2. <a href="#2">Exploratory data analysis</a>
3. <a href="#3">Train-validation dataset split</a>
4. <a href="#4">Text processing and transformation</a>
5. <a href="#5">Using GloVe Word Embeddings</a>
6. <a href="#6">Training and validating model</a>
7. <a href="#7">Improvement ideas</a>

Overall dataset schema:
* __reviewText:__ Text of the review
* __summary:__ Summary of the review
* __verified:__ Whether the purchase was verified (True or False)
* __time:__ UNIX timestamp for the review
* __log_votes:__ Logarithm-adjusted votes log(1+votes)
* __isPositive:__ Whether the review is positive or negative (1 or 0)

__Important note:__ One big distinction betweeen the regular neural networks and RNNs is that RNNs work with sequential data. In our case, RNNs will help us with the text field. If we also want to consider other fields such as time, log_votes, verified, etc. , we need to use the regular neural networks with the RNN network.

In [ ]:
%%capture
%pip install -q -r ../requirements.txt

In [ ]:
import re
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

## 1. <a name="1">Reading the dataset</a>
(<a href="#0">Go to top</a>)

Let's read the dataset below and fill-in the reviewText field. We will use this field as input to our ML model.

In [ ]:
import pandas as pd

df = pd.read_csv('../data/examples/AMAZON-REVIEW-DATA-CLASSIFICATION.csv')

Let's look at the first five rows in the dataset. As you can see the __log_votes__ field is numeric. That's why we will build a regression model.

In [ ]:
df.head()

## 2. <a name="2">Exploratory Data Analysis</a>
(<a href="#0">Go to top</a>)

Let's look at the range and distribution of log_votes

In [ ]:
df["isPositive"].value_counts()

We can check the number of missing values for each columm below.

In [ ]:
print(df.isna().sum())

We have missing values in our text fields.

## 3. <a name="3">Train-validation split</a>
(<a href="#0">Go to top</a>)

Let's split the dataset into training and validation

In [ ]:
# This separates 15% of the entire dataset into validation dataset.
train_text, val_text, train_label, val_label = \
    train_test_split(df["reviewText"].tolist(),
                     df["isPositive"].tolist(),
                     test_size=0.10,
                     shuffle=True,
                     random_state=324)

## 4. <a name="4">Text processing and Transformation</a>
(<a href="#0">Go to top</a>)

We will apply the following processes here:
* __Text cleaning:__ Simple text cleaning operations. We won't do stemming or lemmatization as our word vectors already cover different forms of words. We are using GloVe word embeddings for 6 billion words, phrases or punctuations in this example.
* __Tokenization:__ Tokenizing all sentences
* __Creating vocabulary:__ We will create a vocabulary of the tokens. In this vocabulary, tokens will map to unique ids, such as "car"->32, "house"->651, etc.
* __Transforming text:__ Tokenized sentences will be mapped to unique ids. For example: ["this", "is", "sentence"] -> [13, 54, 412].

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

nltk.download('punkt')
nltk.download('punkt_tab')

def cleanStr(text):
    
    # Check if the sentence is a missing value
    if isinstance(text, str) == False:
        text = ""
            
    # Remove leading/trailing whitespace
    text = text.lower().strip()
    # Remove extra space and tabs
    text = re.sub('\s+', ' ', text)
    # Remove HTML tags/markups
    text = re.compile('<.*?>').sub('', text)
    return text

def tokenize(text):
    tokens = []
    text = cleanStr(text)
    words = word_tokenize(text)
    for word in words:
        tokens.append(word)
    return tokens

def createVocabulary(text_list, min_freq):
    all_tokens = []
    for sentence in text_list:
        all_tokens += tokenize(sentence)
    # Calculate token frequencies
    counter = Counter(all_tokens)
    # Create the vocabulary
    vocab = {'<unk>': 0}
    idx = 1
    for token, count in counter.items():
        if count >= min_freq:
            vocab[token] = idx
            idx += 1
    
    return vocab

def transformText(text, vocab, max_length):
    token_arr = np.zeros((max_length,))
    tokens = tokenize(text)[0:max_length]
    for idx, token in enumerate(tokens):
        token_arr[idx] = vocab.get(token, 0)  # Use 0 for unknown words
    return token_arr

In order to keep the training time low, we only consider the first 250 words (max_length) in sentences. We also only use words that occur more than 5 times in the all sentences (min_freq).

In [ ]:
min_freq = 5
max_length = 250

print("Creating the vocabulary")
vocab = createVocabulary(train_text, min_freq)
print("Transforming training texts")
train_text_transformed = torch.tensor([transformText(text, vocab, max_length) for text in train_text])
print("Transforming validation texts")
val_text_transformed = torch.tensor([transformText(text, vocab, max_length) for text in val_text])

Let's see some unique ids for some words.

In [ ]:
print("Vocabulary index for computer:", vocab.get('computer', 0))
print("Vocabulary index for beautiful:", vocab.get('beautiful', 0))
print("Vocabulary index for code:", vocab.get('code', 0))

## 5. <a name="5">Using pre-trained GloVe Word Embeddings</a>
(<a href="#0">Go to top</a>)

In this example, we will use GloVe word vectors. `'glove.6B.50d.txt'` file gives us 6 billion words/phrases vectors. Each word vector has 50 numbers in it. The following code shows how to get the word vectors and create an embedding matrix from them. We will connect our vocabulary indexes to the GloVe embedding.

In [ ]:
# Download and extract Glove
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

In [ ]:
def load_glove(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
    return words, word_to_vec_map

words, word_to_vec_map = load_glove('glove.6B.50d.txt')

embedding_dim = 50
embedding_matrix = np.zeros((len(vocab), embedding_dim))
for word, i in vocab.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_matrix = torch.FloatTensor(embedding_matrix)

## 6. <a name="6">Training and validation</a>
(<a href="#0">Go to top</a>)

We have processed our text data and also created our embedding matrixes from GloVe. Now, it is time to start the training process.

We will set our parameters below

In [ ]:
# Size of the state vectors
hidden_size = 12

# General NN training parameters
learning_rate = 0.01
epochs = 15
batch_size = 32

# Embedding vector and vocabulary sizes
num_embed = 50 # glove.6B.50d.txt
vocab_size = len(vocab)

We need to put our data into correct format before the process.

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

train_label = torch.tensor(train_label)
val_label = torch.tensor(val_label)

train_dataset = TensorDataset(torch.tensor(train_text_transformed), train_label)
train_loader = DataLoader(train_dataset, batch_size=batch_size)

Our sequential model is made of these layers:
* Embedding layer: This is where our words/tokens are mapped to word vectors.
* RNN layer: We will be using a simple RNN model. We won't stack RNN units in this example. It uses a single RNN unit with its hidden state size of 12.
* Linear layer: A linear layer with a single neuron is used to output our log_votes prediction.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class SimpleRNN(torch.nn.Module):
    def __init__(self, vocab_size, num_embed, hidden_size):
        super(SimpleRNN, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, num_embed)
        self.rnn = torch.nn.RNN(num_embed, hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, 1)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        embedded = self.embedding(x)
        output, _ = self.rnn(embedded)
        output = self.fc(output[:, -1, :])
        return self.sigmoid(output)

model = SimpleRNN(vocab_size, num_embed, hidden_size).to(device)

Let's initialize this network. Then, we will need to make the embedding layer use our GloVe word vectors.

In [ ]:
# Initialize networks parameters
def init_weights(m):
    if type(m) == torch.nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

model.apply(init_weights)

# We set the embedding layer's parameters from GloVe
model.embedding.weight.data.copy_(torch.tensor(embedding_matrix))
# We won't change/train the embedding layer
model.embedding.weight.requires_grad = False

We will define the optimizer and loss function below. __Binary cross-entropy loss__ is used as this is a binary classification problem.
$$
\mathrm{BinaryCrossEntropyLoss} = -\sum_{examples}{(y\log(p) + (1 - y)\log(1 - p))}
$$

In [ ]:
# Setting our optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# We will use Binary Cross-entropy loss
criterion = torch.nn.BCELoss()

Now, it is time to start the training process. We will print the Binary cross-entropy loss after each epoch.

In [ ]:
import time
for epoch in range(epochs):
    start = time.time()
    model.train()
    training_loss = 0.0
    # Training loop
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data.to(torch.long))
        loss = criterion(output.squeeze(), target.float())
        loss.backward()
        optimizer.step()
        
        training_loss += loss.item() * data.size(0)
    
    # Calculate validation loss
    model.eval()
    with torch.no_grad():
        val_predictions = model(torch.tensor(val_text_transformed).to(device).to(torch.long))
        val_loss = criterion(val_predictions.squeeze(), val_label.float().to(device))
    
    # Let's take the average losses
    training_loss = training_loss / len(train_label)
    val_loss = val_loss.item()
    
    end = time.time()
    print(f"Epoch {epoch}. Train_loss {training_loss:.4f} Validation_loss {val_loss:.4f} Seconds {end-start:.2f}")

Let's see some validation results below

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

# Get validation predictions
model.eval()
with torch.no_grad():
    val_predictions = model(torch.tensor(val_text_transformed).to(device).to(torch.long))

# Round predictions: 1 if pred>0.5, 0 otherwise
val_predictions = (val_predictions.squeeze().cpu() > 0.5).float().numpy()

print("Classification Report")
print(classification_report(val_label.numpy(), val_predictions))
print("Accuracy")
print(accuracy_score(val_label.numpy(), val_predictions))

## 7. <a name="7">Improvement ideas</a>
(<a href="#0">Go to top</a>)

We can improve our model by
* Changing hyper-parameters
* Using more advanced architectures such as Gated Recurrent Units (GRU) and Long Short-term Memory networks (LSTM).